# Assignment - Game: UNO

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Run all cells** up to Section 4 (this loads the game client)
2. **Implement your solver** in Section 5
3. **Update STUDENT_TOKEN** in Section 6
4. **Play the game** by running Section 6

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
- You can also use `manual_player_solver` to play manually if you want
- Everything else is provided for you!

### About UNO
UNO is a classic card game:
- Players try to get rid of all their cards first
- Match cards by color or number/symbol
- Special cards: Skip, Reverse, Draw 2, Wild, Wild Draw 4
- Must say "UNO" when you have one card left (automatic)
- Strategic card management and timing is key

---
## Section 1: Setup

**Run this cell (no changes needed)**

In [7]:
import requests
import json
import time
import random
from typing import List, Optional, Tuple, Any, Dict

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'

print("✅ Configuration loaded")

✅ Dependencies imported
✅ Configuration loaded


---
## Section 2: Game Client Library

**Run this cell (no changes needed)**

In [8]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'
 
        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })

        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")

        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)

        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")


def play_game(solver, base_url: str, token: str, game_type: str, game_class,
              multiplayer: bool = False, match_id: Optional[str] = None, 
              num_games: int = 1, debug: bool = False, verbose: bool = True) -> Tuple:
    client = GameClient(base_url, token, debug=debug)

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = losses = draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        game_state = client.get_game_state(match_id, game_num)
        if 'my-player' in game_state:
            player = game_state['my-player']
        
        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)

        while game_state['status'] != 'complete':
            game_state = client.get_game_state(match_id, game_num)
            if 'my-player' in game_state:
                player = game_state['my-player']
            if 'winner' in game_state:
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            if game.is_terminal():
                break

            if verbose:
                game.print_state()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")
                try:
                    move = solver(game)
                    if verbose:
                        print(f"   Move: {move}")
                    client.make_move(match_id, player, move)
                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        if verbose:
            game.print_state()
            print("=" * 40)

        winner = game_state.get('winner')
        if winner == '-':
            if verbose:
                print("🤝 DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST")
            result = 'loss'
            losses += 1

        all_results.append((result, player, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Record: {wins}W - {losses}L - {draws}D")

    return {
        'wins': wins, 'losses': losses, 'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player, 'match_id': match_id
    }, all_results

print("✅ play_game loaded")

✅ GameClient loaded
✅ play_game loaded


---
## Section 3: Game State Class

**Run this cell (no changes needed)**

In [9]:
class UnoGame:
    """Represents UNO game state."""

    def __init__(self, state: str, status: str, current_player: str, my_player: str):
        self.state_str = state
        self.status = status
        self.current_player = current_player
        self.my_player = my_player
        self._state = None

    @property
    def state(self) -> Dict:
        if self._state is None:
            self._state = json.loads(self.state_str)
        return self._state

    def get_my_hand(self) -> List[Dict]:
        return self.state['hands'].get(self.my_player, [])

    def get_hand_sizes(self) -> Dict[str, int]:
        hands = self.state.get('hands', {})
        return {player: len(hand) for player, hand in hands.items()}

    def get_current_color(self) -> str:
        return self.state.get('current_color', '')

    def get_top_card(self) -> Dict:
        discard_pile = self.state.get('discard_pile', [])
        return discard_pile[-1] if discard_pile else {}

    def get_discard_pile(self) -> List[Dict]:
        """Get the full discard pile (all cards that have been played)."""
        return self.state.get('discard_pile', [])

    def get_discard_pile_size(self) -> int:
        """Get the number of cards in the discard pile."""
        return len(self.state.get('discard_pile', []))

    def is_terminal(self) -> bool:
        return self.status == 'complete'

    def _can_play_card(self, card: Dict, top_card: Dict, current_color: str) -> bool:
        if not top_card:
            return True
        if card.get('type') == 'wild':
            return True
        if card.get('color') == current_color:
            return True
        if card.get('value') == top_card.get('value'):
            return True
        return False

    def get_valid_moves(self) -> List[Dict]:
        if self.current_player != self.my_player:
            return []
        
        hand = self.get_my_hand()
        top_card = self.get_top_card()
        current_color = self.get_current_color()
        valid_moves = []
        
        for i, card in enumerate(hand):
            if self._can_play_card(card, top_card, current_color):
                move = {'type': 'play', 'card_index': i, 'card': card, 'call_uno': len(hand) == 2}
                if card.get('type') == 'wild':
                    for color in ['red', 'blue', 'green', 'yellow']:
                        color_move = move.copy()
                        color_move['color_choice'] = color
                        valid_moves.append(color_move)
                else:
                    valid_moves.append(move)
        
        valid_moves.append({'type': 'draw', 'count': 1})
        return valid_moves

    def print_state(self):
        print(f"\n{'='*50}")
        print(f"Current Turn: Player {self.current_player}")
        print(f"Current Color: {self.get_current_color().upper()}")
        top_card = self.get_top_card()
        if top_card:
            print(f"Top Card: {top_card.get('color')} {top_card.get('value')}")
        hand_sizes = self.get_hand_sizes()
        print("\nHand Sizes:")
        for p in sorted(hand_sizes.keys()):
            if p != self.my_player:
                print(f"  Player {p}: {hand_sizes[p]} cards")
        my_hand = self.get_my_hand()
        print(f"\nYour Hand ({len(my_hand)} cards):")
        for i, card in enumerate(my_hand):
            print(f"  {i}: {card.get('color')} {card.get('value')}")
        print('='*50)

print("✅ UnoGame class loaded")

✅ UnoGame class loaded


---
## Section 4: Manual solver

**Run this cell (no changes needed)**

In [10]:
def manual_player_solver(game: UnoGame) -> Dict:
    """
    Interactive manual player - YOU choose your moves!
    """
    game.print_state()
    
    valid_moves = game.get_valid_moves()
    
    if not valid_moves:
        print("No valid moves!")
        return {'type': 'draw', 'count': 1}
    
    print(f"\n🎮 YOUR TURN (Player {game.my_player})!")
    print("\nValid moves:")
    
    move_list = []
    move_idx = 0
    
    # Display playable cards
    play_moves = [m for m in valid_moves if m.get('type') == 'play']
    if play_moves:
        print("\n🎴 Cards you can play:")
        for move in play_moves:
            card_idx = move.get('card_index')
            card = move.get('card', {})
            color = card.get('color', '?').upper()
            value = card.get('value', '?')
            
            if card.get('type') == 'wild':
                choice = move.get('color_choice', 'red').upper()
                print(f"  {move_idx}: Play WILD as {choice}")
            else:
                print(f"  {move_idx}: Play {color} {value} (card #{card_idx})")
            
            move_list.append(move)
            move_idx += 1
    
    # Display draw option
    print(f"\n  {move_idx}: Draw a card")
    move_list.append({'type': 'draw', 'count': 1})
    
    # Get player choice
    while True:
        try:
            choice = input(f"\nEnter move number (0-{len(move_list)-1}, or 'q' to quit): ").strip()
            
            if choice.lower() == 'q':
                raise KeyboardInterrupt()
            
            idx = int(choice)
            if 0 <= idx < len(move_list):
                selected_move = move_list[idx]
                
                # If it's a play move with a wild card, ask for color choice
                if selected_move.get('type') == 'play' and selected_move.get('card', {}).get('type') == 'wild':
                    print("\nWild card color choices:")
                    colors = ['red', 'blue', 'green', 'yellow']
                    for i, c in enumerate(colors):
                        print(f"  {i}: {c.upper()}")
                    
                    while True:
                        try:
                            color_choice = input("Choose color (0-3): ").strip()
                            color_idx = int(color_choice)
                            if 0 <= color_idx < len(colors):
                                selected_move['color_choice'] = colors[color_idx]
                                break
                            else:
                                print(f"❌ Invalid choice! Enter 0-{len(colors)-1}")
                        except ValueError:
                            print("❌ Invalid input! Enter a number.")
                
                return selected_move
            else:
                print(f"❌ Invalid index! Choose 0-{len(move_list)-1}")
        
        except ValueError:
            print("❌ Invalid input! Enter a number or 'q' to quit.")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise

print("✅ Manual player solver loaded")

✅ Manual player solver loaded


---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

### Available Methods

```python
game.get_my_hand()                        # List of cards in your hand
game.get_hand_sizes()                     # Dict of player hand sizes
game.get_current_color()                  # Current active color
game.get_top_card()                       # Card on top of discard pile
game.get_discard_pile()                   # Full discard pile history (all played cards)
game.get_discard_pile_size()              # Number of cards in discard pile
game.get_valid_moves()                    # All valid moves you can make
game.is_terminal()                        # Whether game is finished
game.print_state()                        # Print current game state
```

### Move Format
- Play a card: `{'type': 'play', 'card_index': i, 'card': {...}, 'color_choice': 'red'}`
  - `card_index`: Index of card in your hand (0-indexed)
  - `color_choice`: Only required for Wild cards (one of: 'red', 'blue', 'green', 'yellow')
- Draw a card: `{'type': 'draw', 'count': 1}`

### Card Information
Card dict contains: `{'color': 'red', 'value': '5', 'type': 'normal'}`
- Colors: 'red', 'blue', 'green', 'yellow'
- Values: '0'-'9', 'skip', 'reverse', 'draw2'
- Types: 'normal', 'action', 'wild'

### Strategic Tips
- **Discard Pile History**: Use `game.get_discard_pile()` to see all cards that have been played. This allows you to implement card-counting strategies!
- **Example**: Count which high cards are still in the deck vs. already played to make better decisions
- **Hand Tracking**: Use `game.get_hand_sizes()` to track how many cards opponents have

In [11]:
def my_agent(game: UnoGame) -> dict:
    """
    Strong heuristic UNO agent.

    Key ideas:
    - Game phases (early / mid / late) with different behavior.
    - Aggressive vs opponents with 1–2 cards (use draw2 / wild_draw4 / skip / reverse).
    - Keep wilds for late game unless they give a big swing.
    - Use discard pile to bias wild color choices toward colors we have
      and that are underrepresented in discard (others less likely to have).
    """

    import random as _random

    valid_moves = game.get_valid_moves()
    if not valid_moves:
        # No valid moves returned -> must draw
        return {'type': 'draw', 'count': 1}

    # Separate play vs draw moves
    play_moves = [m for m in valid_moves if m.get('type') == 'play']
    if not play_moves:
        # Only draw is possible
        return {'type': 'draw', 'count': 1}

    hand = game.get_my_hand()
    hand_size = len(hand)

    colors = ['red', 'blue', 'green', 'yellow']

    # ---- GAME PHASE ----
    if hand_size >= 8:
        phase = 'early'
    elif hand_size >= 4:
        phase = 'mid'
    else:
        phase = 'late'

    # ---- COLOR COUNTS IN HAND ----
    color_counts = {c: 0 for c in colors}
    for c in hand:
        col = c.get('color')
        if col in color_counts:
            color_counts[col] += 1

    # Our "preferred" color (best for chaining in the future)
    preferred_color = max(colors, key=lambda c: color_counts[c])

    # ---- GLOBAL STATE INFO ----
    top_card = game.get_top_card()
    current_color = game.get_current_color()

    hand_sizes = game.get_hand_sizes()  # {player_id: size}
    my_player_id = game.my_player
    opponent_sizes = [sz for pid, sz in hand_sizes.items() if pid != my_player_id]

    min_opp = min(opponent_sizes) if opponent_sizes else None
    avg_opp = sum(opponent_sizes) / len(opponent_sizes) if opponent_sizes else None

    # ---- DISCARD PILE COLOR COUNTS ----
    discard = game.get_discard_pile()
    discard_color_counts = {c: 0 for c in colors}
    for d in discard:
        dc = d.get('color')
        if dc in discard_color_counts:
            discard_color_counts[dc] += 1

    # ------------------------------------------------------------------
    # HELPER: best color to choose when playing a wild
    # ------------------------------------------------------------------
    def choose_wild_color(exclude_index=None) -> str:
        """
        Choose color for a wild:
        - Prioritize colors we have many of (after playing this wild).
        - Penalize colors heavily present in discard (others more likely to be out/holding them).
        - When an opponent is low on cards, bias away from the current_color if possible.
        """
        # Remaining cards in hand if we play the card at exclude_index
        remaining_counts = {c: 0 for c in colors}
        for i, card in enumerate(hand):
            if i == exclude_index:
                continue
            col = card.get('color')
            if col in remaining_counts:
                remaining_counts[col] += 1

        def score_color(col: str) -> float:
            score = 0.0
            # More cards of that color in our hand is good
            score += remaining_counts[col] * 4.0
            # Fewer cards of that color in discard is good (less likely others have)
            score -= discard_color_counts.get(col, 0) * 1.0
            # If an opponent is very low, prefer changing color away from current_color
            if min_opp is not None and min_opp <= 2:
                if col != current_color:
                    score += 3.0
                else:
                    score -= 2.0
            return score

        # If all remaining_counts are 0 (e.g., only wilds left), just use least-played color
        if sum(remaining_counts.values()) == 0:
            return min(colors, key=lambda c: discard_color_counts.get(c, 0))

        return max(colors, key=score_color)

    # ------------------------------------------------------------------
    # HELPER: evaluate a single move
    # ------------------------------------------------------------------
    def score_move(move: dict) -> float:
        # Draw moves are always worse than having any playable card,
        # and we call score_move only for play moves.
        if move.get('type') != 'play':
            return -1e9

        score = 0.0
        card = move.get('card', {}) or {}
        card_type = card.get('type')
        value = str(card.get('value'))

        # Resulting hand size after playing this card
        resulting_hand_size = hand_size - 1

        # --------------------------------------------------------------
        # 1. GAME ENDING / HAND SIZE EFFECT
        # --------------------------------------------------------------
        if resulting_hand_size == 0:
            # Immediate win is the ultimate priority
            score += 10000.0
        elif resulting_hand_size == 1:
            # Getting to UNO is extremely good (especially mid/late game)
            score += 2500.0 if phase == 'late' else 1500.0

        # General preference: smaller hand size
        score += (hand_size - resulting_hand_size) * 50.0

        # --------------------------------------------------------------
        # 2. CARD TYPE IMPORTANCE
        # --------------------------------------------------------------
        is_wild_draw4 = ('draw4' in value.lower())

        # Final color after play:
        # - For wild: color_choice
        # - For others: card color
        move_color = move.get('color_choice', card.get('color'))

        # ACTION / WILD / NUMBER
        if card_type == 'wild':
            # Wild baseline strength depends on phase:
            if phase == 'early':
                # Mostly keep wilds early; weaker baseline
                score += 5.0
            elif phase == 'mid':
                score += 40.0
            else:  # late
                score += 80.0

            # Wild draw 4 is very strong
            if is_wild_draw4:
                score += 120.0
                if min_opp is not None and min_opp <= 2:
                    # Hammer low-card opponents with draw4
                    score += 200.0

            # Synergy: choose colors where we still have many cards after playing this one
            remaining_counts = color_counts.copy()
            idx = move.get('card_index')
            if isinstance(idx, int) and 0 <= idx < len(hand):
                this_color = hand[idx].get('color')
                if this_color in remaining_counts:
                    remaining_counts[this_color] -= 1
            score += remaining_counts.get(move_color, 0) * 25.0

            # Penalize choosing colors that are heavy in discard
            score -= discard_color_counts.get(move_color, 0) * 3.0

            # If we would otherwise have no playable non-wild card on current color/value,
            # using wild gets extra value (avoids drawing)
            has_non_wild_reply = any(
                h.get('type') != 'wild' and
                (h.get('color') == current_color or h.get('value') == top_card.get('value'))
                for h in hand
            )
            if not has_non_wild_reply:
                score += 30.0

        elif card_type == 'action':
            # Action cards (skip / reverse / draw2)
            # Base importance; bigger later in the game
            score += 40.0 if phase == 'early' else 65.0

            if value == 'draw2':
                score += 90.0
                # Very strong against low-card opponents
                if min_opp is not None and min_opp <= 2:
                    score += 180.0
                if avg_opp is not None and avg_opp <= 3:
                    score += 40.0
            elif value in ('skip', 'reverse'):
                score += 45.0
                if min_opp is not None and min_opp <= 2:
                    # Skipping a near-UNO opponent is huge
                    score += 90.0

        else:
            # Number / normal cards: less powerful but good for dumping
            score += 25.0
            # Slight bonus for matching top value (keeps options)
            if top_card and value == str(top_card.get('value')):
                score += 10.0

        # --------------------------------------------------------------
        # 3. COLOR / FUTURE PLAYABILITY
        # --------------------------------------------------------------
        # If we keep the same color, we maintain tempo
        if move_color == current_color:
            score += 20.0

        # Moving to our preferred color is good (we likely can chain)
        if move_color == preferred_color:
            score += 10.0

        # Remove singleton colors from our hand (simplifies color management)
        card_color = card.get('color')
        if card_color in colors:
            if color_counts[card_color] == 1:
                # Getting rid of a "weird" single color is useful
                score += 25.0
            elif color_counts[card_color] >= 3:
                # Slight penalty for burning too many of a strong color
                score -= 5.0

        # When an opponent is low, try to change color away from current
        if min_opp is not None and min_opp <= 2:
            if move_color != current_color:
                score += 15.0
            else:
                score -= 5.0

        # --------------------------------------------------------------
        # 4. LATE GAME PREFERENCES
        # --------------------------------------------------------------
        if phase == 'late':
            # In late game, keeping wilds as finishers is valuable;
            # but we already gave wilds a strong base, so:
            if card_type != 'wild':
                score += 20.0  # prefer dumping non-wilds first
            if card_type == 'number':
                # Number cards are usually "safe" in late game
                score += 10.0

        # --------------------------------------------------------------
        # 5. SMALL RANDOM NOISE TO BREAK TIES
        # --------------------------------------------------------------
        score += _random.random() * 1e-3

        return score

    # ------------------------------------------------------------------
    # PREPROCESS: ensure wild play moves always have a color_choice
    # ------------------------------------------------------------------
    scored_moves = []
    for m in play_moves:
        mv = dict(m)  # shallow copy
        card = mv.get('card', {}) or {}
        if card.get('type') == 'wild' and 'color_choice' not in mv:
            mv['color_choice'] = choose_wild_color(exclude_index=mv.get('card_index'))
        s = score_move(mv)
        scored_moves.append((s, mv))

    # ------------------------------------------------------------------
    # CHOOSE BEST MOVE
    # ------------------------------------------------------------------
    scored_moves.sort(key=lambda x: x[0], reverse=True)
    best_score = scored_moves[0][0]

    # Allow very near-best alternatives for some diversity
    EPSILON = 0.5  # moves within 0.5 of best_score are considered equivalent
    candidate_moves = [mv for s, mv in scored_moves if s >= best_score - EPSILON]

    # Tie-breaker: prefer moves that more aggressively reduce hand size
    def tie_break_key(mv):
        card = mv.get('card', {}) or {}
        ctype = card.get('type')
        # All are play moves so resulting_hand_size = hand_size - 1
        resulting_hand_size = hand_size - 1
        # Prefer action over number in tie
        is_action = 1 if ctype == 'action' else 0
        is_wild = 1 if ctype == 'wild' else 0
        # We want: smaller resulting hand, then action, then wild, etc.
        # sort() is ascending, so use negative flags for desired order.
        return (resulting_hand_size, -is_action, -is_wild)

    candidate_moves.sort(key=tie_break_key)
    chosen = _random.choice(candidate_moves)

    # ------------------------------------------------------------------
    # MAP BACK TO ONE OF THE ORIGINAL valid_moves JUST TO BE SAFE
    # (especially for wilds / color_choice variants)
    # ------------------------------------------------------------------
    def map_to_valid(chosen_move, valid_list):
        if chosen_move.get('type') == 'draw':
            for v in valid_list:
                if v.get('type') == 'draw':
                    return v
            return {'type': 'draw', 'count': 1}

        if chosen_move.get('type') == 'play':
            idx = chosen_move.get('card_index')
            for v in valid_list:
                if v.get('type') != 'play':
                    continue
                if v.get('card_index') != idx:
                    continue
                v_card = v.get('card', {}) or {}
                if v_card.get('type') == 'wild':
                    # Prefer same color_choice if exists
                    if 'color_choice' in chosen_move and 'color_choice' in v:
                        if v['color_choice'] == chosen_move['color_choice']:
                            return v
                    # Otherwise, just use this variant
                    return v
                else:
                    # Non-wild cards: card_index match is enough
                    return v
        return None

    mapped = map_to_valid(chosen, valid_moves)
    if mapped:
        return mapped

    # Fallback: any play move with same card_index
    chosen_idx = chosen.get('card_index')
    for v in valid_moves:
        if v.get('type') == 'play' and v.get('card_index') == chosen_idx:
            return v

    # Absolute last resort
    return {'type': 'draw', 'count': 1}


---
## Section 6: Play the Game!

**Update STUDENT_TOKEN below and run to play**

You can choose which solver to use:
- `my_agent` - Your AI implementation (default)
- `manual_player_solver` - Interactive manual play

In [12]:
STUDENT_TOKEN = 'Juan Sebastian Peña'
SOLVER = my_agent  # Change to manual_player_solver to play manually!
MULTIPLAYER = False
MATCH_ID = None
NUM_GAMES = 50

try:
    stats, results = play_game(
        solver=SOLVER,
        base_url=BASE_URL,
        token=STUDENT_TOKEN,
        game_type='uno4',
        game_class=UnoGame,
        multiplayer=MULTIPLAYER,
        num_games=NUM_GAMES,
        match_id=MATCH_ID,
        verbose=True
    )

    print("\n📊 Summary:")
    print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
    print(f"   Win Rate: {stats['win_rate']*100:.1f}%")

except Exception as e:
    print(f"❌ Game error: {e}")
    import traceback
    traceback.print_exc()

🎮 Creating new match: 50 x uno4
   Match ID: 2566
🔗 Joining match 2566...
   You are player: 1

🎮 GAME 1/50


Current Turn: Player 4
Current Color: RED
Top Card: red draw2

Hand Sizes:
  Player 1: 7 cards
  Player 2: 7 cards
  Player 3: 7 cards

Your Hand (7 cards):
  0: blue 2
  1: wild wild_draw4
  2: yellow 6
  3: blue draw2
  4: red 6
  5: yellow 1
  6: red 2
🤔 Your turn (Player 4)...
   Move: {'type': 'play', 'card_index': 1, 'card': {'color': 'wild', 'value': 'wild_draw4', 'type': 'wild'}, 'call_uno': False, 'color_choice': 'red'}

Current Turn: Player 1
Current Color: YELLOW
Top Card: wild wild_draw4

Hand Sizes:
  Player 1: 11 cards
  Player 2: 6 cards
  Player 3: 6 cards

Your Hand (10 cards):
  0: blue 2
  1: yellow 6
  2: blue draw2
  3: red 6
  4: yellow 1
  5: red 2
  6: yellow 0
  7: blue 2
  8: blue 5
  9: wild wild_draw4
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 4
Current Color: YELLOW
Top Card: yellow skip

Hand Sizes:
  Player 1: 10 cards
  Player 2: 